In [1]:
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge beautifulsoup4

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  53.19 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  36.54 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  39.27 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  46.69 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |###################

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
import requests

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from bs4 import BeautifulSoup

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

from urllib.request import urlopen
import requests
import ssl

print('Libraries imported.')

Libraries imported.


In [3]:
context = ssl._create_unverified_context()
wikiurl = 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto'
html = urlopen(wikiurl, context=context)
bsObj = BeautifulSoup(html.read(), 'lxml')

In [8]:
neighborhoodItems = bsObj.find_all('div', {'style': 'padding:0em 0.25em'})[:6]
boItems = bsObj.find_all('th', {'class': 'navbox-group'})[:6]
boList = []
for item in boItems:
    boList.append(item.find('a').contents[0])
nbDict = {}
for i in range(6):
    nbDict[boList[i]] = [[x.contents[0], x.get('href')] for x in neighborhoodItems[i].find_all('a')]
temp = []
for item in nbDict.keys():
    for neigh in nbDict[item]:
        if neigh in temp:
            nbDict[item].remove(neigh)
        else:
            temp.append(neigh)
#print(nbDict)

In [9]:
wikiurl = 'https://en.wikipedia.org'
latlonDict = {}
for i in nbDict.keys():
    for j in range(len(nbDict[i])):
        nbUrl = wikiurl+str(nbDict[i][j][1])
        nbhtml = urlopen(nbUrl, context=context)
        nbbsObj = BeautifulSoup(nbhtml.read(), 'lxml')
        try:
            latlon = nbbsObj.find('span', {'class': 'geo'}).contents[0]
            latlonDict[nbDict[i][j][0]]=[float(x) for x in latlon.split('; ')]
        except:
            continue
print(latlonDict)

{'The Junction': [43.66556, -79.46444], 'Crescent Town': [43.69583, -79.29306], 'Yonge–Eglinton': [43.706802, -79.398271], 'Harbord Village': [43.661, -79.406], 'Trinity–Bellwoods': [43.6467, -79.4083], 'North Toronto': [43.711, -79.389], 'Richview': [43.67722, -79.55083], 'Birch Cliff': [43.6875, -79.26944], 'Teddington Park': [43.734184, -79.400962], 'The Queensway – Humber Bay': [43.63028, -79.48417], 'Baby Point': [43.6575, -79.4925], 'Financial District': [43.64793472, -79.38175222], 'Lawrence Heights': [43.719, -79.45], 'Moss Park': [43.655709, -79.367316], 'West Don Lands': [43.655, -79.353], 'Rexdale': [43.72194, -79.57194], 'Bayview Woods – Steeles': [43.797, -79.383], 'Swansea': [43.64389, -79.47778], 'Weston': [43.7009889, -79.5197], 'Leslieville': [43.661927, -79.332039], 'Alderwood': [43.6075, -79.54028], 'Lytton Park': [43.717829, -79.407833], 'South Hill': [43.681, -79.404], 'Upper Beaches': [43.682088, -79.303272], 'Cabbagetown': [43.6664, -79.3629], 'Liberty Village': 

In [10]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [13]:
for borough in nbDict.keys():
    for j in range(len(nbDict[borough])):
        try:
            neighborhoods = neighborhoods.append({'Borough': borough, 'Neighborhood': nbDict[borough][j][0], \
                                                  'Latitude': latlonDict[nbDict[borough][j][0]][0], \
                                                  'Longitude': latlonDict[nbDict[borough][j][0]][1]}, ignore_index=True)
        except:
            #print(borough, nbDict[borough][j][0])
            continue

In [14]:
neighborhoods.shape

(245, 4)

In [15]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Old Toronto,Agincourt,43.79250,-79.28389
1,Old Toronto,Armadale,43.83722,-79.25806
2,Old Toronto,Bendale,43.76000,-79.24000
3,Old Toronto,Birch Cliff,43.68750,-79.26944
4,Old Toronto,Brown's Corners,43.81222,-79.24667


In [16]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 6 boroughs and 245 neighborhoods.


In [17]:
address = 'Bendale, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bendale are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bendale are 43.7535196, -79.2553355.


In [19]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [20]:
CLIENT_ID = 'XDXJHQDLIGUDBLI5MKZUK41X24KIUVF23ZZC4I0K4AAC3NYE' # your Foursquare ID
CLIENT_SECRET = '44JFLQQRP04JKKM4KT5UOUHYNRK4TXQNMBDP2TV1ARJ4R4JB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XDXJHQDLIGUDBLI5MKZUK41X24KIUVF23ZZC4I0K4AAC3NYE
CLIENT_SECRET:44JFLQQRP04JKKM4KT5UOUHYNRK4TXQNMBDP2TV1ARJ4R4JB


In [21]:
neighborhoods.loc[0, 'Neighborhood']

'Agincourt'

In [22]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Agincourt are 43.7925, -79.28389.


In [23]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=XDXJHQDLIGUDBLI5MKZUK41X24KIUVF23ZZC4I0K4AAC3NYE&client_secret=44JFLQQRP04JKKM4KT5UOUHYNRK4TXQNMBDP2TV1ARJ4R4JB&v=20180605&ll=43.7925,-79.28389&radius=500&limit=100'

In [24]:
results = requests.get(url).json()

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Donalda Park,Park,43.790280,-79.283630
1,Havendale Park,Park,43.796010,-79.284398
2,LT turf,BBQ Joint,43.788811,-79.281426


In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
# type your answer here
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )


Agincourt
Armadale
Bendale
Birch Cliff
Brown's Corners
Clairlea
Cliffcrest
Cliffside
Dorset Park
Eglinton East
Golden Mile
Guildwood
Highland Creek
Ionview
L'Amoreaux
Malvern
Maryvale
Milliken
Morningside
Morningside Heights
Oakridge
Port Union
Rouge
Scarborough City Centre
Scarborough Junction
Scarborough Village
Alexandra Park
Allenby
The Annex
The Beaches
Bedford Park
Bickford Park
Bloor West Village
Bloor Street Culture Corridor
Bloordale Village
Bracondale Hill
Brockton Village
Cabbagetown
Carleton Village
Casa Loma
Chaplin Estates
Chinatown
Christie Pits
Church and Wellesley
CityPlace
Corktown
Corso Italia
Davenport
Davisville Village
Deer Park
Discovery District
Alexandra Park
Allenby
The Annex
The Beaches
Bedford Park
Bickford Park
Bloor West Village
Bloor Street Culture Corridor
Bloordale Village
Bracondale Hill
Brockton Village
Cabbagetown
Carleton Village
Casa Loma
Chaplin Estates
Chinatown
Christie Pits
Church and Wellesley
CityPlace
Corktown
Corso Italia
Davenport
Davisvil

In [31]:
print(toronto_venues.shape)
toronto_venues.head()

(6537, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt,43.79250,-79.28389,Donalda Park,43.790280,-79.283630,Park
1,Agincourt,43.79250,-79.28389,Havendale Park,43.796010,-79.284398,Park
2,Agincourt,43.79250,-79.28389,LT turf,43.788811,-79.281426,BBQ Joint
3,Armadale,43.83722,-79.25806,China Cottage,43.835215,-79.254603,Hakka Restaurant
4,Armadale,43.83722,-79.25806,LCBO,43.835178,-79.256168,Liquor Store


In [32]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,6,6,6,6,6,6
Alderwood,4,4,4,4,4,4
Alexandra Park,200,200,200,200,200,200
Allenby,10,10,10,10,10,10
Amesbury,3,3,3,3,3,3
Armadale,24,24,24,24,24,24
Armour Heights,1,1,1,1,1,1
Baby Point,12,12,12,12,12,12
Bathurst Manor,4,4,4,4,4,4


In [33]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 324 uniques categories.


In [34]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Pla

In [35]:
toronto_onehot.shape

(6537, 324)

In [36]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Pla

In [37]:
toronto_grouped.shape

(190, 324)

In [38]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                     venue  freq
0                     Park  0.67
1                BBQ Joint  0.33
2              Yoga Studio  0.00
3  New American Restaurant  0.00
4             Optical Shop  0.00


----Alderwood----
                venue  freq
0          Playground  0.25
1                Park  0.25
2  Athletics & Sports  0.25
3              Market  0.25
4         Yoga Studio  0.00


----Alexandra Park----
                           venue  freq
0                            Bar  0.06
1                           Café  0.05
2  Vegetarian / Vegan Restaurant  0.05
3                    Coffee Shop  0.04
4                   Dessert Shop  0.03


----Allenby----
                     venue  freq
0  Health & Beauty Service   0.2
1            Moving Target   0.2
2           Ice Cream Shop   0.2
3         Business Service   0.2
4                   Garden   0.2


----Amesbury----
                 venue  freq
0        Grocery Store  0.33
1          Supermarket  0.33
2              

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Park,BBQ Joint,Women's Store,Farmers Market,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farm
1,Alderwood,Playground,Market,Athletics & Sports,Park,Women's Store,Farmers Market,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space
2,Alexandra Park,Bar,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Cocktail Bar,French Restaurant,Dessert Shop,Sandwich Place,Restaurant,Record Shop
3,Allenby,Garden,Moving Target,Ice Cream Shop,Business Service,Health & Beauty Service,Women's Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space
4,Amesbury,Grocery Store,Gym,Supermarket,Women's Store,Farm,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space


In [41]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 1, 4, 4, 4, 4, 1, 3, 4], dtype=int32)

In [42]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Old Toronto,Agincourt,43.79250,-79.28389,3.0,Park,BBQ Joint,Women's Store,Farmers Market,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farm
1,Old Toronto,Armadale,43.83722,-79.25806,4.0,Bubble Tea Shop,Hakka Restaurant,Coffee Shop,Fast Food Restaurant,Burger Joint,Spa,Liquor Store,Big Box Store,Juice Bar,Greek Restaurant
2,Old Toronto,Bendale,43.76000,-79.24000,4.0,Diner,Fast Food Restaurant,Pizza Place,Chinese Restaurant,Farmers Market,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space
3,Old Toronto,Birch Cliff,43.68750,-79.26944,4.0,Diner,Discount Store,Bank,Thai Restaurant,Flower Shop,Farm,Elementary School,Food & Drink Shop,Empanada Restaurant,Ethiopian Restaurant
4,Old Toronto,Brown's Corners,43.81222,-79.24667,4.0,Train Station,Vietnamese Restaurant,Chocolate Shop,Gym,Fast Food Restaurant,Sandwich Place,Afghan Restaurant,Fish Market,Event Service,Flower Shop


In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)

['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']


In [49]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    try:
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster-1)],
            fill=True,
            fill_color=rainbow[int(cluster-1)],
            fill_opacity=0.7).add_to(map_clusters)
    except:
        continue
map_clusters

In [50]:
# CLuster 1

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Morningside Heights,Grocery Store,Pizza Place,Women's Store,Farmers Market,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant
30,Bedford Park,Juice Bar,Gift Shop,Pizza Place,Farm,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space
55,Bedford Park,Juice Bar,Gift Shop,Pizza Place,Farm,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space
147,Rockcliffe–Smythe,Pizza Place,Bus Line,Furniture / Home Store,Women's Store,Fast Food Restaurant,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant
161,Eatonville,Bar,Home Service,Pizza Place,Farmers Market,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space
173,Princess Gardens,Playground,Pizza Place,Women's Store,Farm,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space
185,Clanton Park,Women's Store,Wings Joint,Pizza Place,American Restaurant,Mediterranean Restaurant,Bus Line,Fast Food Restaurant,Ethiopian Restaurant,Event Service,Event Space
195,Humber Summit,Bakery,Empanada Restaurant,Pizza Place,Women's Store,Fast Food Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farm
233,Morningside Heights,Grocery Store,Pizza Place,Women's Store,Farmers Market,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant


In [51]:
# Cluster 2

toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Guildwood,Grocery Store,Coffee Shop,Pizza Place,Sports Bar,Sandwich Place,Bus Line,Park,Women's Store,Elementary School,Empanada Restaurant
13,Ionview,Coffee Shop,Chinese Restaurant,Pizza Place,Sandwich Place,Bank,Grocery Store,Intersection,Discount Store,Event Space,Elementary School
15,Malvern,Bar,Caribbean Restaurant,Women's Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market
23,Scarborough City Centre,Clothing Store,Coffee Shop,Cosmetics Shop,Tea Room,Sandwich Place,Movie Theater,Wings Joint,Food Court,Sporting Goods Shop,Discount Store
26,Alexandra Park,Bar,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Cocktail Bar,French Restaurant,Dessert Shop,Sandwich Place,Restaurant,Record Shop
28,The Annex,Coffee Shop,Greek Restaurant,Pizza Place,Park,Thai Restaurant,Pub,Gym,Grocery Store,Clothing Store,Restaurant
29,The Beaches,Bakery,Beach,Bar,Thai Restaurant,Nail Salon,Tea Room,Pizza Place,Coffee Shop,Japanese Restaurant,Park
31,Bickford Park,Italian Restaurant,Bar,Café,Beer Bar,American Restaurant,Burrito Place,Dessert Shop,Japanese Restaurant,Coffee Shop,Gastropub
32,Bloor West Village,Café,Coffee Shop,Bakery,Falafel Restaurant,Pizza Place,Restaurant,Pub,Tea Room,Shoe Store,Italian Restaurant
33,Bloor Street Culture Corridor,Coffee Shop,Cosmetics Shop,Café,Pub,Gym,Restaurant,Boutique,Sporting Goods Shop,Park,Italian Restaurant


In [52]:
# Cluster 3

toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
184,Bridle Path,Music Venue,Food Court,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market


In [53]:
# Cluster 4

toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Park,BBQ Joint,Women's Store,Farmers Market,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farm
24,Scarborough Junction,Train Station,Sandwich Place,Fast Food Restaurant,Park,Women's Store,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Service
104,Moore Park,Park,Beer Store,Women's Store,Farmers Market,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farm
120,Rosedale,Park,Playground,Trail,Women's Store,Falafel Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Service
125,South Hill,Park,Skating Rink,History Museum,Historic Site,French Restaurant,Garden Center,Athletics & Sports,Women's Store,Elementary School,Empanada Restaurant
128,Swansea,Park,Restaurant,Pilates Studio,Skating Rink,Falafel Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Service
152,Crescent Town,Metro Station,Convenience Store,Park,Golf Course,Women's Store,Farm,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Service
153,Governor's Bridge,Cosmetics Shop,Playground,Park,Trail,Women's Store,Farm,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Service
159,Alderwood,Playground,Market,Athletics & Sports,Park,Women's Store,Farmers Market,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space
164,Humber Heights – Westmount,Pizza Place,Park,Women's Store,Farm,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space


In [54]:
# Cluster 5

toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Armadale,Bubble Tea Shop,Hakka Restaurant,Coffee Shop,Fast Food Restaurant,Burger Joint,Spa,Liquor Store,Big Box Store,Juice Bar,Greek Restaurant
2,Bendale,Diner,Fast Food Restaurant,Pizza Place,Chinese Restaurant,Farmers Market,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space
3,Birch Cliff,Diner,Discount Store,Bank,Thai Restaurant,Flower Shop,Farm,Elementary School,Food & Drink Shop,Empanada Restaurant,Ethiopian Restaurant
4,Brown's Corners,Train Station,Vietnamese Restaurant,Chocolate Shop,Gym,Fast Food Restaurant,Sandwich Place,Afghan Restaurant,Fish Market,Event Service,Flower Shop
5,Clairlea,Convenience Store,Fast Food Restaurant,Diner,Mexican Restaurant,Sandwich Place,Grocery Store,Women's Store,Falafel Restaurant,Elementary School,Empanada Restaurant
6,Cliffcrest,Fast Food Restaurant,Liquor Store,Discount Store,Burger Joint,Pizza Place,Pharmacy,Park,Empanada Restaurant,Ethiopian Restaurant,Event Service
7,Cliffside,Pizza Place,Breakfast Spot,Sandwich Place,Pub,Grocery Store,Gastropub,Women's Store,Falafel Restaurant,Elementary School,Empanada Restaurant
8,Dorset Park,Fast Food Restaurant,Breakfast Spot,Gym / Fitness Center,Burrito Place,Sri Lankan Restaurant,Furniture / Home Store,Light Rail Station,Sushi Restaurant,Grocery Store,Gym
9,Eglinton East,Pharmacy,Pizza Place,Wings Joint,Bus Station,Breakfast Spot,Bowling Alley,Japanese Restaurant,Rock Club,Grocery Store,Liquor Store
10,Golden Mile,Fast Food Restaurant,Coffee Shop,Clothing Store,Shopping Plaza,Shoe Store,Pet Store,Cosmetics Shop,Health Food Store,Furniture / Home Store,Japanese Restaurant
